<a href="https://colab.research.google.com/github/kgndirangu/project-4/blob/main/optimization_dummies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,073 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,889 kB]
Get:13 http://security.ubuntu.com/ubuntu ja

In [2]:
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [3]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Read in data from S3 Buckets

url = "https://project4-ny-health-data-1.s3.amazonaws.com/inpatient_hospital_2.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("inpatient_hospital_2.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show Spark DataFrame
df.show()

+-------------------+---------------+-----------+--------------------+---------+-------------------+------+-----+-----------------+--------------+-----------------+-------------------+------------------+-------------------------+------------------+-------------------------+------------+--------------------+------------+--------------------+----------------------------+-----------------------------------+---------------------+--------------------------------+-------------------+------------------------------+-------------+-----------+--------+-------------+--------+-------------+---------+-------------+-------+-------------+---------+-------------+--------+-------------+---------+--------------+---------+-------------+---------+-------------+
|Health Service Area|Hospital County|Facility ID|       Facility Name|Age Group|Zip Code - 3 digits|Gender| Race|        Ethnicity|Length of Stay|Type of Admission|Patient Disposition|CCS Diagnosis Code|CCS Diagnosis Description|CCS Procedure Code|

In [ ]:
df.dtypes

[('Health Service Area', 'string'),
 ('Hospital County', 'string'),
 ('Facility ID', 'double'),
 ('Facility Name', 'string'),
 ('Age Group', 'string'),
 ('Zip Code - 3 digits', 'string'),
 ('Gender', 'string'),
 ('Race', 'string'),
 ('Ethnicity', 'string'),
 ('Length of Stay', 'string'),
 ('Type of Admission', 'string'),
 ('Patient Disposition', 'string'),
 ('CCS Diagnosis Code', 'double'),
 ('CCS Diagnosis Description', 'string'),
 ('CCS Procedure Code', 'double'),
 ('CCS Procedure Description', 'string'),
 ('APR DRG Code', 'int'),
 ('APR DRG Description', 'string'),
 ('APR MDC Code', 'int'),
 ('APR MDC Description', 'string'),
 ('APR Severity of Illness Code', 'int'),
 ('APR Severity of Illness Description', 'string'),
 ('APR Risk of Mortality', 'string'),
 ('APR Medical Surgical Description', 'string'),
 ('Source of Payment 1', 'string'),
 ('Emergency Department Indicator', 'string'),
 ('Total Charges', 'double'),
 ('Total Costs', 'double'),
 ('AgeIndex', 'double'),
 ('catVecAge', '

In [23]:
#Drop vector variables
df_no_vec = df.drop("Health Service Area", "Hospital County", "Facility Name", "catVecAge", "catVecSex", "catVecRace", "catVecER", "catVecPay1", "catVecAdm","catVecDisp","catVecMort", "catVecSurg",
                    "AgeIndex", "SexIndex", "RaceIndex", "ERIndex", "Pay1Index", "AdmIndex", "DispIndex", "MortIndex", "SurgIndex", "Zip Code - 3 digits", "CCS Diagnosis Description", "APR DRG Description",
                    "APR MDC Description", "APR Severity of Illness Description", "Total Costs", "CCS Procedure Description")

In [24]:
df_no_vec.show()

+-----------+---------+------+-----+-----------------+--------------+-----------------+-------------------+------------------+------------------+------------+------------+----------------------------+---------------------+--------------------------------+-------------------+------------------------------+-------------+
|Facility ID|Age Group|Gender| Race|        Ethnicity|Length of Stay|Type of Admission|Patient Disposition|CCS Diagnosis Code|CCS Procedure Code|APR DRG Code|APR MDC Code|APR Severity of Illness Code|APR Risk of Mortality|APR Medical Surgical Description|Source of Payment 1|Emergency Department Indicator|Total Charges|
+-----------+---------+------+-----+-----------------+--------------+-----------------+-------------------+------------------+------------------+------------+------------+----------------------------+---------------------+--------------------------------+-------------------+------------------------------+-------------+
|     1437.0|  0 to 17|     M|White|N

In [25]:
df_no_vec.withColumn("Length of Stay",col("Length of Stay").cast("int"))

DataFrame[Facility ID: double, Age Group: string, Gender: string, Race: string, Ethnicity: string, Length of Stay: int, Type of Admission: string, Patient Disposition: string, CCS Diagnosis Code: double, CCS Procedure Code: double, APR DRG Code: int, APR MDC Code: int, APR Severity of Illness Code: int, APR Risk of Mortality: string, APR Medical Surgical Description: string, Source of Payment 1: string, Emergency Department Indicator: string, Total Charges: double]

In [43]:
pandas_df = df_no_vec.toPandas()

In [44]:
pandas_df["Length of Stay"] = pandas_df["Length of Stay"].replace("120 +", 121)

In [45]:
pandas_df["Length of Stay"] = pandas_df["Length of Stay"].astype(int)

In [46]:
pandas_df.dtypes

Facility ID                         float64
Age Group                            object
Gender                               object
Race                                 object
Ethnicity                            object
Length of Stay                        int64
Type of Admission                    object
Patient Disposition                  object
CCS Diagnosis Code                  float64
CCS Procedure Code                  float64
APR DRG Code                          int32
APR MDC Code                          int32
APR Severity of Illness Code          int32
APR Risk of Mortality                object
APR Medical Surgical Description     object
Source of Payment 1                  object
Emergency Department Indicator       object
Total Charges                       float64
dtype: object

In [47]:
pandas_df["Length of Stay"].unique()

array([  2,   3,   5,   4,  19,   1,  10,  13,  37,  27,  34,  26,   6,
        32,  12,  14,  18,   7,  20,  51,   8,  80,  11,   9,  57, 121,
        25,  29,  24,  33,  17,  23,  21,  15,  30,  28,  84,  36,  40,
        16,  52, 102,  41,  64,  22,  65,  31,  55,  38,  46,  45,  58,
        35,  69,  59,  49,  95,  43,  47,  61,  50,  42,  48,  94,  71,
        53,  97,  75,  91,  86,  67,  60,  68,  78, 106,  63,  44,  39,
        62,  83, 117,  90, 101,  99,  96, 112,  70,  79, 109,  77,  56,
       103,  98,  54,  85,  89,  72,  73,  74,  92, 113,  81,  88,  87,
        93,  76, 104, 115, 107, 118,  66, 114, 100, 105, 108, 116, 110,
        82, 111, 119])

In [48]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(pandas_df)
dummies_df

,Facility ID,Length of Stay,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,Total Charges,Age Group_0 to 17,Age Group_18 to 29,...,Source of Payment 1_Insurance Company,Source of Payment 1_Medicaid,Source of Payment 1_Medicare,Source of Payment 1_Other Federal Program,Source of Payment 1_Other Non-Federal Program,Source of Payment 1_Self-Pay,Source of Payment 1_Unknown,Source of Payment 1_Workers Compensation,Emergency Department Indicator_N,Emergency Department Indicator_Y
0,1437.0,2,218.0,228.0,640,15,1,2468.95,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1437.0,3,218.0,228.0,640,15,1,6647.18,1,0,...,0,0,0,0,0,0,0,0,1,0
2,1437.0,2,218.0,228.0,640,15,1,5702.18,1,0,...,1,0,0,0,0,0,0,0,1,0
3,1437.0,2,218.0,228.0,640,15,1,2125.58,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1437.0,3,218.0,220.0,640,15,1,3050.58,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443897,3975.0,5,245.0,203.0,204,5,2,36663.34,0,0,...,0,0,1,0,0,0,0,0,1,0
443898,3975.0,25,99.0,177.0,468,11,2,156389.24,0,0,...,0,0,1,0,0,0,0,0,1,0
443899,3975.0,6,14.0,78.0,221,6,2,53022.38,0,0,...,0,0,1,0,0,0,0,0,1,0
443900,3975.0,3,96.0,193.0,200,5,3,18968.74,0,0,...,0,0,0,0,0,1,0,0,1,0


In [49]:
# Split our preprocessed data into our features and target arrays

y = dummies_df["Total Charges"].values
X = dummies_df.drop("Total Charges", axis = 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [50]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="linear", input_dim=input_features))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="sigmoid"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                5200      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7661 (29.93 KB)
Trainable params: 7661 (29.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:

# Compile the model
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#https://stackoverflow.com/questions/33846069/how-to-set-rmse-cost-function-in-tensorflow
#mae=mean absolute error
rmse = tf.keras.metrics.RootMeanSquaredError()
nn.compile(optimizer="sgd", loss='mean_squared_error',
              metrics=[rmse,'mae'])



In [53]:
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
#Save weights every 5 epochs
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="",
    #period=5,
    save_freq = 5
  )

In [62]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
10404/10404 [==============================] - 18s 2ms/step - loss: 6382319616.0000 - root_mean_squared_error: 79889.4219 - mae: 35077.3008
Epoch 2/50
10404/10404 [==============================] - 19s 2ms/step - loss: 6382765056.0000 - root_mean_squared_error: 79892.2109 - mae: 35063.6719
Epoch 3/50
10404/10404 [==============================] - 18s 2ms/step - loss: 6384033792.0000 - root_mean_squared_error: 79900.1484 - mae: 35060.5117
Epoch 4/50
10404/10404 [==============================] - 19s 2ms/step - loss: 6384711680.0000 - root_mean_squared_error: 79904.3906 - mae: 35067.4414
Epoch 5/50
10404/10404 [==============================] - 19s 2ms/step - loss: 6382331904.0000 - root_mean_squared_error: 79889.5000 - mae: 35067.7930
Epoch 6/50
10404/10404 [==============================] - 20s 2ms/step - loss: 6382307840.0000 - root_mean_squared_error: 79889.3438 - mae: 35070.2734
Epoch 7/50
10404/10404 [==============================] - 18s 2ms/step - loss: 6384224768.0000

In [63]:
# Evaluate the model using the test data
model_loss, model_rmse, model_mae = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, RMSE: {model_rmse}, mae: {model_mae}")

3468/3468 - 4s - loss: 8033117696.0000 - root_mean_squared_error: 89627.6641 - mae: 32419.5234 - 4s/epoch - 1ms/step
Loss: 8033117696.0, RMSE: 89627.6640625, mae: 32419.5234375


In [64]:
#https://www.tutorialspoint.com/keras/keras_regression_prediction_using_mpl.htm
prediction = nn.predict(X_test_scaled)
#prediction = prediction.flatten()
prediction

3468/3468 [==============================] - 5s 1ms/step


array([[30497.582],
       [56360.188],
       [16927.684],
       ...,
       [56360.188],
       [43743.867],
       [49376.438]], dtype=float32)

In [65]:
prediction.flatten()

array([30497.582, 56360.188, 16927.684, ..., 56360.188, 43743.867,
       49376.438], dtype=float32)

In [66]:
y_test_reshape = y_test.reshape(-1, 1)
y_test_reshape

array([[ 2884.7 ],
       [21864.75],
       [36315.99],
       ...,
       [ 4706.05],
       [16616.55],
       [24419.22]])

In [67]:
#https://www.tensorflow.org/api_docs/python/tf/keras/metrics/R2Score
metric = tf.keras.metrics.R2Score()
metric.update_state(y_test_reshape, prediction)
result = metric.result()
print(result)
result.numpy()

tf.Tensor(0.020110726, shape=(), dtype=float32)


0.020110726

Optimization attempt 1

In [60]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=12)
dt.fit(X_train, y_train)
predictions = dt.predict(X_test_scaled)
dt.score(X_test_scaled, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


-0.09255875126363233

Optimization attempt 2

In [61]:
#Try linear regression machine learning
#https://www.turing.com/kb/scikit-learn-cheatsheet-methods-for-classification-and-regression
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
predictions = lr.predict(X_test_scaled)
lr.score(X_test_scaled, y_test)

0.5280448979133505

In [ ]:
#Alternate optimizers
#https://www.tensorflow.org/api_docs/python/tf/keras/optimizers